# Example for running signal - background classification

See $ROOTSYS/tutorials/tmva/TMVAClassification.C for more details.
First get the path to the classes. The environment variable HSCODE must be set in your shell setup file (.bashrc or .tchsrc,....)
 Load/compile the hsmva classes via ROOTs ACLiC (clang-based compiler)


In [1]:
gROOT->ProcessLine(".x $HSCODE/hsmva/LoadHSMVA.C+");

Get the TMVA tutorial data files and load them into signal and background trees

In [2]:
%%cpp -d
#include "GetTutorialFile.h"


In [3]:
TFile* input = GetClassificationFile();

auto *signalTree     = dynamic_cast<TTree*>(input->Get("TreeS"));
auto *background     = dynamic_cast<TTree*>(input->Get("TreeB"));


--- TMVAClassification       : Using input file: ./files/tmva_class_example.root


Info in <TFile::OpenFromCache>: using local cache copy of http://root.cern.ch/files/tmva_class_example.root [./files/tmva_class_example.root]


Create and configure the training.
Must give an output directory for storing trees and training weight files. This will also be linked to the result

In [4]:
TrainSignalID train("TMVAClassificationTut");

train.SetOutDir("/work/dump"); //Set this to a sensible output directory
train.IgnoreBranches("");//Any branches in tree not used must be flagged!
train.AddSignalTree(signalTree); //Add signal tree
train.AddBackgroundTree(background); //Add background tree, branches must be same as signal
train.PrepareTrees(); //make test and training trees


DataSetInfo              : [TMVAClassificationTut] : Added class "Signal"
                         : Add Tree TreeS of type Signal with 6000 events
DataSetInfo              : [TMVAClassificationTut] : Added class "Background"
                         : Add Tree TreeB of type Background with 6000 events


Book the MVA classifier we want to use.
Either configuring by string or using predefined method from MethConf.h

In [10]:
  //Can Book methods either via standard TMVA::Factory interface...
  train.BookMethod(TMVA::Types::kBDT, "BDT","!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20");

  //..or predefined methods (See HSMVA::MethConf.h)
  train.BookMethod(HS::MVA::Meths.MLP);


Factory                  : Booking method: BDT
                         : 
DataSetFactory           : [TMVAClassificationTut] : Number of events in input trees
                         : 
                         : 
                         : Number of training and testing events
                         : ---------------------------------------------------------------------------
                         : Signal     -- training events            : 1000
                         : Signal     -- testing events             : 1000
                         : Signal     -- training and testing events: 2000
                         : Background -- training events            : 1000
                         : Background -- testing events             : 1000
                         : Background -- training and testing events: 2000
                         : 
DataSetInfo              : Correlation matrix (Signal):
                         : ----------------------------------------
              

And run the training!

In [ ]:
train.DoTraining();

In [7]:
%jsroot on

Now make some plots. Note more diagnostics can be found in the Training.root file in the output directory.
This includes the trees with the training and testing variables. This can also be analysed in standard ROOT
running using TMVAGUI

In [8]:
train.DrawROCCurve();


input_line_96:2:7: error: member reference type 'HS::MVA::TrainSignalID' is not a pointer; did you mean to
      use '.'?
 train->DrawROCCurve();
 ~~~~~^~
      .


In [9]:
train.DrawResponses();

input_line_97:2:7: error: member reference type 'HS::MVA::TrainSignalID' is not a pointer; did you mean to
      use '.'?
 train->DrawResponses();
 ~~~~~^~
      .


Now you can try filling a tree with the resulting response. 
The new tree can contain different data to the original trees, but must have the same branches.
Link to [MVA result](tutSignalIDResult.ipynb)